## Setup

In [6]:
# Import Library

import os
import pandas as pd
from xgboost import XGBRegressor


DATA_DIR = "data"
SUBMISSION_DIR = "submission"
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH = os.path.join(DATA_DIR, "test_case.csv")
SUBMISSION_PATH = os.path.join(DATA_DIR, "sample_submission.csv")
TARGET = "apparent_temperature (°C)"

"""
Change filename up to your preference
"""
SUBMISSION_NAME = "submission_1.csv" 
SUBMISSION_FILE = os.path.join(SUBMISSION_DIR, SUBMISSION_NAME)

In [7]:
# Load Data

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

## Exploratory Data Analysis

In [8]:
# Data Exploration
""" 
you can fill this section with your code for data exploration
"""
# Example
train.sample(10)

,time,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),apparent_temperature (°C),pressure_msl (hPa),surface_pressure (hPa),snowfall (cm),cloudcover (%),cloudcover_low (%),...,soil_temperature_0_to_7cm (°C),soil_temperature_7_to_28cm (°C),soil_temperature_28_to_100cm (°C),soil_temperature_100_to_255cm (°C),soil_moisture_0_to_7cm (m³/m³),soil_moisture_7_to_28cm (m³/m³),soil_moisture_28_to_100cm (m³/m³),soil_moisture_100_to_255cm (m³/m³),city,id
157725,2019-04-10T12:00,30.6,66.0,23.4,39.6,1010.5,1012.2,0.0,45.0,4.0,...,30.7,27.4,27.1,26.7,0.372,0.362,0.312,0.397,lh,169812
93406,2019-06-18T18:00,27.8,87.0,25.4,36.0,1008.8,1010.5,0.0,94.0,31.0,...,29.4,28.5,27.5,27.2,0.422,0.412,0.425,0.402,lh,171474
157360,2020-07-24T11:00,27.7,70.0,21.8,33.4,1012.2,989.2,0.0,30.0,5.0,...,28.0,25.2,25.9,26.2,0.158,0.135,0.156,0.251,si,54083
116458,2019-06-03T21:00,27.1,85.0,24.4,33.7,1009.6,1011.3,0.0,62.0,3.0,...,26.6,27.5,27.1,27.1,0.441,0.433,0.425,0.388,lh,171117
228484,2018-12-29T12:00,29.6,71.0,23.7,34.2,1007.3,982.2,0.0,55.0,16.0,...,28.0,27.3,27.3,27.4,0.096,0.099,0.103,0.082,u,72108
170492,2019-08-08T12:00,28.6,63.0,20.8,33.9,1012.0,989.1,0.0,41.0,17.0,...,29.7,25.0,25.3,25.8,0.077,0.080,0.125,0.240,si,45660
176883,2018-10-05T07:00,27.7,67.0,20.9,31.6,1012.6,1014.9,0.0,38.0,9.0,...,29.1,31.9,31.7,30.4,0.057,0.059,0.071,0.201,su,6655
34790,2018-04-19T22:00,26.4,86.0,23.8,34.3,1010.0,1012.3,0.0,17.0,3.0,...,27.6,29.5,29.1,28.6,0.257,0.256,0.206,0.296,su,2614
104598,2019-05-30T17:00,30.5,55.0,20.6,34.6,1007.6,1009.9,0.0,20.0,15.0,...,33.7,31.3,29.6,28.9,0.059,0.105,0.159,0.274,su,12353
226411,2021-03-03T12:00,28.8,71.0,22.9,35.2,1013.0,1014.7,0.0,44.0,11.0,...,29.2,26.5,26.4,26.0,0.300,0.284,0.339,0.423,lh,186444


## Preprocessing

In [9]:
# Data Preprocessing
"""
you can fill this section with your code for data preprocessing
"""
# Example
train = train.select_dtypes(exclude=['object'])
train.dropna(axis=0, inplace=True)

test = test.select_dtypes(exclude=['object'])

## Modeling

In [10]:
# Modeling using
"""
Change the code just if you need to
"""
train_x = train.drop(columns=[TARGET])
train_y = train[TARGET]

rfr = XGBRegressor()
rfr.fit(train_x, train_y)

predict = rfr.predict(test)
submission[TARGET] = predict

submission.to_csv(SUBMISSION_FILE, index=False)